In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
import numpy as np

In [2]:
url = 'https://www.indeed.com/jobs?q=Data+Science&l=Los+Angeles%2C+CA&vjk=74cadd03cb0dd918'
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get(url)

In [11]:
# all the scraping



job_titles = []
company_names = []
company_ratings = []
company_locations = []
job_pays = []
descriptions = []
posted = []
current_page = 1

pagination = driver.find_element(By.XPATH, ".//nav[contains(@role, 'navigation')]")
last_page = pagination.find_elements(By.XPATH, ".//a[contains(@data-testid, 'pagination-page-5')]")[-1].text
driver.set_page_load_timeout(10)

for page_num in range(1, int(last_page) + 18):
    container = driver.find_element(By.XPATH, ".//div[contains(@id, 'mosaic-jobResults')]")
    all_jobs = container.find_elements(By.XPATH, ".//div[contains(@class, 'job_seen_beacon')]")
    for job in all_jobs:
        title = job.find_element(By.XPATH, ".//h2[contains(@class, 'jobTitle')]").text
        try:
            location = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[1]
        except:
            location = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[0]
        company = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[0]
        when_posted= job.find_element(By.XPATH, ".//tr[contains(@class, 'underShelfFooter')]").text.split("\n")[-1].strip("More...")
        description = job.find_element(By.XPATH, ".//tr[contains(@class, 'underShelfFooter')]").text.split("\n")[0]
        try:
            string = job.find_element(By.XPATH, ".//div[contains(@class, 'salary-snippet-container')]").text
            listy = string.split()
            for element in listy:
                if "$" in element:
                    pay = element
        except:
            pay = "NaN"
        job_titles.append(title)
        job_pays.append(pay)
        company_names.append(company)
        descriptions.append(description)
        company_locations.append(location)
        posted.append(when_posted)
    page_num += 1

    try:
        #next = driver.find_element(By.XPATH, ".//a[contains(@aria-label, 'Next Page')]")
        next = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, ".//a[contains(@aria-label, 'Next Page')]")))
        print(next)
        next.click()
    except:
        #next = False
        print(f'failed {page_num}')
        break

MaxRetryError: HTTPConnectionPool(host='localhost', port=61014): Max retries exceeded with url: /session/c642b3b6c03192d46ec66390582f19ac/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x127b04310>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [4]:
# all the data cleaning
#separate into city and state
cities = []
states = []
# Parse location data
for location in company_locations:
    parts = location.split(', ')
    if len(parts) > 1:
        city = parts[0].replace("Hybrid", "")
        city = city.replace("remote", '')
        city = city.replace('in', '')
        state = parts[1] if len(parts) > 1 else None
        cities.append(city)
        states.append(state[:2])
    else:
        cities.append(np.nan)
        states.append(np.nan)


#make salaries integers
salaries = []
for salary in job_pays:
    if salary == "NaN" or salary == "" :
        salaries.append(np.nan)
    else:
        salary = salary.replace('$', '')
        salary = salary.replace(',', '')
        salary_list = salary.split('.')
        if len(salary_list[0]) <= 3:
            #Then I know it is hourly rate
            salaries.append(float(salary) * 40 * 52)
        else:
            salaries.append(float(salary))


job_info = pd.DataFrame({"Title": job_titles,
              "Company": company_names,
              "Location": company_locations,
              "Job Description": descriptions,
              "When Posted": posted,
              "Pay": salaries,
              "City":cities,
               "State": states})

#try getting hybrid or remote status
def is_hybrid_remote(location):
    return 'Hybrid' in location or 'Remote' in location

# Apply the function to create a new column 'Is_Hybrid_Remote'
job_info['Is_Hybrid_Remote'] = job_info['Location'].apply(is_hybrid_remote)

# fix locations and add ratings column
ratings = []
def fix_location(value):
    try:
        rating = float(value)
        ratings.append(rating)
        return np.nan
    except:
        ratings.append(np.nan)
        return value
job_info['Location'] = job_info["Location"].apply(fix_location)
job_info['Rating'] = ratings


In [8]:
job_info.to_csv('3_24.csv')

In [ ]:
job_info = pd.read_csv('masterpiece_fixed_pay.csv')

In [10]:
driver.quit()

In [34]:
#job_info.to_csv("masterpiece_fixed_pay.csv")